<h3> prøver å gjenskape henning sin 143.</h3>
<p><p>

In [1]:
%load_ext autoreload

In [2]:
from sklearn.model_selection import KFold
import optuna
optuna.logging.set_verbosity(optuna.logging.INFO)

from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor
import pandas as pd
import numpy as np

/Users/miksx/.pyenv/versions/3.11.4/envs/machinelearning/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
%autoreload
from pipeline import Pipeline
pipin = Pipeline()

Current working directory: /Users/miksx/GitHub/Forest-Gump/mikael/optuna


In [4]:
df1_0 = pipin.get_data("A")
df2_0 = pipin.get_data("B")
df3_0 = pipin.get_data("C")

In [5]:
# remove_consecutive_measurments
# 6/24
# dette skal prøve å gjenskape henning sin
df1_0 = pipin.remove_consecutive_measurments_new(df1_0, 4, 12)
df2_0 = pipin.remove_consecutive_measurments_new(df2_0, 4, 12)
df3_0 = pipin.remove_consecutive_measurments_new(df3_0, 4, 12)

In [6]:
df1_0 = pipin.drop_columns(df1_0)
df2_0 = pipin.drop_columns(df2_0)
df3_0 = pipin.drop_columns(df3_0)
print(df1_0.shape, df2_0.shape, df3_0.shape)

(23231, 47) (15838, 47) (13238, 47)


In [7]:
df1_0.head()

,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,diffuse_rad_1h:J,...,super_cooled_liquid_water:kgm2,t_1000hPa:K,total_cloud_cover:p,visibility:m,wind_speed_10m:ms,estimated,observed,day_of_year,hour,pv_measurement
0,7.700,1.22825,1728.949951,0.000000,0.00,1728.949951,0.0,280.299988,0.000,0.000000,...,0.0,286.225006,100.000000,40386.476562,3.600,0.0,1.0,153.0,22.0,0.00
1,7.700,1.22350,1689.824951,0.000000,0.00,1689.824951,0.0,280.299988,0.000,0.000000,...,0.0,286.899994,100.000000,33770.648438,3.350,0.0,1.0,153.0,23.0,0.00
2,7.875,1.21975,1563.224976,0.000000,0.00,1563.224976,0.0,280.649994,0.000,0.000000,...,0.0,286.950012,100.000000,13595.500000,3.050,0.0,1.0,154.0,0.0,0.00
3,8.425,1.21800,1283.425049,208.649994,0.75,1283.425049,0.0,281.674988,0.300,526.775024,...,0.0,286.750000,100.000000,2321.850098,2.725,0.0,1.0,154.0,1.0,0.00
4,8.950,1.21800,1003.500000,32468.150391,23.10,1003.500000,0.0,282.500000,11.975,22068.949219,...,0.0,286.450012,99.224998,11634.799805,2.550,0.0,1.0,154.0,2.0,19.36


In [8]:
def get_objective(X=None, y=None):
    def objective(trial):
        kf = KFold(n_splits=10, shuffle=True, random_state=42)
        objective_list_reg = ["reg:squarederror", "reg:squaredlogerror"]
        tree_method = ["approx", "hist"]

        params = {
            "eval_metric": "mae",
            "objective": "reg:absoluteerror",
            "tree_method": trial.suggest_categorical("tree_method", tree_method),
            "max_depth": trial.suggest_int("max_depth", 3, 10),  # Adjust the range
            "learning_rate": trial.suggest_float("learning_rate", 0.01, 1.0),
            "n_estimators": trial.suggest_int("n_estimators", 50, 1000),
            "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),  # Increase the range
            "gamma": trial.suggest_float("gamma", 0.1, 1.0),  # Increase the lower bound
            "subsample": trial.suggest_discrete_uniform("subsample", 0.6, 1.0, 0.05),  # Reduce the range
            "colsample_bytree": trial.suggest_discrete_uniform("colsample_bytree", 0.6, 1.0, 0.05),  # Reduce the range
            "reg_alpha": trial.suggest_float("reg_alpha", 0.01, 1.0),
            "reg_lambda": trial.suggest_float("reg_lambda", 0.01, 1.0),
            "random_state": trial.suggest_int("random_state", 1, 1000),
        }
        # Initialize a list to store the mean absolute errors for each fold
        mae_list = []

        for fold, (train_idx, val_idx) in enumerate(kf.split(X), 1):
            X_fold_train, X_fold_val = X.iloc[train_idx], X.iloc[val_idx]
            y_fold_train, y_fold_val = y.iloc[train_idx], y.iloc[val_idx]

            print(f"Training Fold {fold}...")
            model = XGBRegressor(**params)
            model.fit(X_fold_train, y_fold_train)
            predictions = model.predict(X_fold_val)
            mae_fold = mean_absolute_error(y_fold_val, predictions)
            mae_list.append(mae_fold)
            print(f"Fold {fold} MAE: {mae_fold}")

        # Calculate the mean of the mean absolute errors for all folds
        mae = sum(mae_list) / len(mae_list)
        print(f"Mean MAE for all Folds: {mae}")
        return mae
    return objective

In [9]:
X_1, y_1 = df1_0[[c for c in df1_0.columns if "pv_measurement" != c]], df1_0["pv_measurement"]

In [10]:
study1 = optuna.create_study(direction="minimize")

# Optimize the objective function
print("Optimizing hyperparameters...")
study1.optimize(get_objective(X=X_1,y=y_1 ), n_trials=30)
print("Optimization complete!")

predictor1 = XGBRegressor(**study1.best_params).fit(X_1, y_1)


[I 2023-11-09 11:39:18,295] A new study created in memory with name: no-name-9495fb70-1f3f-405c-9b22-955b9cac2cf4


Optimizing hyperparameters...
Training Fold 1...


/var/folders/y8/9glnt_6110xftbsjpdqd12280000gp/T/ipykernel_3270/1140765383.py:16: FutureWarning: suggest_discrete_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., step=...) instead.
  "subsample": trial.suggest_discrete_uniform("subsample", 0.6, 1.0, 0.05),  # Reduce the range
/var/folders/y8/9glnt_6110xftbsjpdqd12280000gp/T/ipykernel_3270/1140765383.py:17: FutureWarning: suggest_discrete_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., step=...) instead.
  "colsample_bytree": trial.suggest_discrete_uniform("colsample_bytree", 0.6, 1.0, 0.05),  # Reduce the range


Fold 1 MAE: 315.2620259549906
Training Fold 2...
Fold 2 MAE: 321.0163103734126
Training Fold 3...
Fold 3 MAE: 320.90846119891
Training Fold 4...
Fold 4 MAE: 315.4512398263404
Training Fold 5...
Fold 5 MAE: 316.5879135496964
Training Fold 6...
Fold 6 MAE: 314.9845747085011
Training Fold 7...
Fold 7 MAE: 328.07706254338837
Training Fold 8...
Fold 8 MAE: 306.02335009781115
Training Fold 9...
Fold 9 MAE: 318.6807757530847
Training Fold 10...


[I 2023-11-09 11:40:02,800] Trial 0 finished with value: 317.67355333438184 and parameters: {'tree_method': 'hist', 'max_depth': 7, 'learning_rate': 0.9751687172235494, 'n_estimators': 316, 'min_child_weight': 9, 'gamma': 0.3456179054224886, 'subsample': 0.9, 'colsample_bytree': 0.85, 'reg_alpha': 0.5175898760800687, 'reg_lambda': 0.40495170580753903, 'random_state': 597}. Best is trial 0 with value: 317.67355333438184.


...

Fold 10 MAE: 245.96222454629253
Mean MAE for all Folds: 246.55063017224583
Training Fold 1...


/var/folders/y8/9glnt_6110xftbsjpdqd12280000gp/T/ipykernel_3270/1140765383.py:16: FutureWarning: suggest_discrete_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., step=...) instead.
  "subsample": trial.suggest_discrete_uniform("subsample", 0.6, 1.0, 0.05),  # Reduce the range
/var/folders/y8/9glnt_6110xftbsjpdqd12280000gp/T/ipykernel_3270/1140765383.py:17: FutureWarning: suggest_discrete_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., step=...) instead.
  "colsample_bytree": trial.suggest_discrete_uniform("colsample_bytree", 0.6, 1.0, 0.05),  # Reduce the range


Fold 1 MAE: 256.08711456578305
Training Fold 2...
Fold 2 MAE: 266.44881651745385
Training Fold 3...
Fold 3 MAE: 251.5267766706845
Training Fold 4...
Fold 4 MAE: 257.13107070523796
Training Fold 5...
Fold 5 MAE: 267.42026345699213
Training Fold 6...
Fold 6 MAE: 263.84761283029286
Training Fold 7...
Fold 7 MAE: 261.13894897582804
Training Fold 8...
Fold 8 MAE: 252.72501011409332
Training Fold 9...
Fold 9 MAE: 275.06149525665353
Training Fold 10...


[I 2023-11-09 12:44:31,267] Trial 29 finished with value: 261.40118542122525 and parameters: {'tree_method': 'hist', 'max_depth': 7, 'learning_rate': 0.31551951108383686, 'n_estimators': 717, 'min_child_weight': 4, 'gamma': 0.352538290137301, 'subsample': 0.85, 'colsample_bytree': 0.8, 'reg_alpha': 0.47756105180371716, 'reg_lambda': 0.674078845191346, 'random_state': 231}. Best is trial 23 with value: 245.49391084430485.


Fold 10 MAE: 262.6247451192333
Mean MAE for all Folds: 261.40118542122525
Optimization complete!


In [11]:
X_2, y_2 = df2_0[[c for c in df2_0.columns if "pv_measurement" != c]], df2_0["pv_measurement"]

In [12]:
study2 = optuna.create_study(direction="minimize")

# Optimize the objective function
print("Optimizing hyperparameters...")
study2.optimize(get_objective(X=X_2,y=y_2 ), n_trials=30)
print("Optimization complete!")
predictor2 = XGBRegressor(**study2.best_params).fit(X_2, y_2)


[I 2023-11-09 12:44:37,336] A new study created in memory with name: no-name-6d9c5b7b-d3c2-4567-893e-729293c85982
/var/folders/y8/9glnt_6110xftbsjpdqd12280000gp/T/ipykernel_3270/1140765383.py:16: FutureWarning: suggest_discrete_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., step=...) instead.
  "subsample": trial.suggest_discrete_uniform("subsample", 0.6, 1.0, 0.05),  # Reduce the range
/var/folders/y8/9glnt_6110xftbsjpdqd12280000gp/T/ipykernel_3270/1140765383.py:17: FutureWarning: suggest_discrete_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., step=...) instead.
  "colsample_bytree": trial.suggest_discrete_uniform("colsample_bytree", 0.6, 1.0, 0.05),  # Reduce the range


Optimizing hyperparameters...
Training Fold 1...
Fold 1 MAE: 47.818905619852615
Training Fold 2...
Fold 2 MAE: 40.99605185371725
Training Fold 3...
Fold 3 MAE: 40.83230637785472
Training Fold 4...
Fold 4 MAE: 44.117366828501076
Training Fold 5...
Fold 5 MAE: 44.746223185309645
Training Fold 6...
Fold 6 MAE: 44.73780386376908
Training Fold 7...
Fold 7 MAE: 44.900588346857
Training Fold 8...
Fold 8 MAE: 43.52663494904538
Training Fold 9...
Fold 9 MAE: 45.02981395459154
Training Fold 10...


[I 2023-11-09 12:44:48,524] Trial 0 finished with value: 44.15456502281014 and parameters: {'tree_method': 'hist', 'max_depth': 5, 'learning_rate': 0.2633383644116773, 'n_estimators': 243, 'min_child_weight': 8, 'gamma': 0.46292481405748465, 'subsample': 0.8, 'colsample_bytree': 0.6, 'reg_alpha': 0.41652034361955287, 'reg_lambda': 0.8954073610978595, 'random_state': 588}. Best is trial 0 with value: 44.15456502281014.


Fold 10 MAE: 44.83995524860315
Mean MAE for all Folds: 44.15456502281014
Training Fold 1...


/var/folders/y8/9glnt_6110xftbsjpdqd12280000gp/T/ipykernel_3270/1140765383.py:16: FutureWarning: suggest_discrete_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., step=...) instead.
  "subsample": trial.suggest_discrete_uniform("subsample", 0.6, 1.0, 0.05),  # Reduce the range
/var/folders/y8/9glnt_6110xftbsjpdqd12280000gp/T/ipykernel_3270/1140765383.py:17: FutureWarning: suggest_discrete_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., step=...) instead.
  "colsample_bytree": trial.suggest_discrete_uniform("colsample_bytree", 0.6, 1.0, 0.05),  # Reduce the range


...

Fold 10 MAE: 41.1936670813104
Mean MAE for all Folds: 40.53854346835023
Training Fold 1...


/var/folders/y8/9glnt_6110xftbsjpdqd12280000gp/T/ipykernel_3270/1140765383.py:16: FutureWarning: suggest_discrete_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., step=...) instead.
  "subsample": trial.suggest_discrete_uniform("subsample", 0.6, 1.0, 0.05),  # Reduce the range
/var/folders/y8/9glnt_6110xftbsjpdqd12280000gp/T/ipykernel_3270/1140765383.py:17: FutureWarning: suggest_discrete_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., step=...) instead.
  "colsample_bytree": trial.suggest_discrete_uniform("colsample_bytree", 0.6, 1.0, 0.05),  # Reduce the range


Fold 1 MAE: 49.83942385762127
Training Fold 2...
Fold 2 MAE: 42.072135661659686
Training Fold 3...
Fold 3 MAE: 42.53062241820694
Training Fold 4...
Fold 4 MAE: 45.08120576281028
Training Fold 5...
Fold 5 MAE: 45.85079370750816
Training Fold 6...
Fold 6 MAE: 45.70737973205824
Training Fold 7...
Fold 7 MAE: 44.80473873398167
Training Fold 8...
Fold 8 MAE: 43.569585776032184
Training Fold 9...
Fold 9 MAE: 46.084278354649165
Training Fold 10...


[I 2023-11-09 13:45:23,245] Trial 29 finished with value: 45.23629356582783 and parameters: {'tree_method': 'approx', 'max_depth': 4, 'learning_rate': 0.25132261351546015, 'n_estimators': 243, 'min_child_weight': 7, 'gamma': 0.25484567370710365, 'subsample': 0.8, 'colsample_bytree': 0.8, 'reg_alpha': 0.09720382195734925, 'reg_lambda': 0.90773229771827, 'random_state': 809}. Best is trial 7 with value: 39.1855482651186.


Fold 10 MAE: 46.822771653750735
Mean MAE for all Folds: 45.23629356582783
Optimization complete!


In [13]:
X_3, y_3 = df3_0[[c for c in df3_0.columns if "pv_measurement" != c]], df3_0["pv_measurement"]

In [14]:
study3 = optuna.create_study(direction="minimize")

# Optimize the objective function
print("Optimizing hyperparameters...")
study3.optimize(get_objective(X=X_3,y=y_3 ), n_trials=30)
print("Optimization complete!")
predictor3 = XGBRegressor(**study3.best_params).fit(X_3, y_3)


[I 2023-11-09 13:45:29,569] A new study created in memory with name: no-name-723d3e9e-0f2f-42b7-9829-09846eda1b59
/var/folders/y8/9glnt_6110xftbsjpdqd12280000gp/T/ipykernel_3270/1140765383.py:16: FutureWarning: suggest_discrete_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., step=...) instead.
  "subsample": trial.suggest_discrete_uniform("subsample", 0.6, 1.0, 0.05),  # Reduce the range
/var/folders/y8/9glnt_6110xftbsjpdqd12280000gp/T/ipykernel_3270/1140765383.py:17: FutureWarning: suggest_discrete_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., step=...) instead.
  "colsample_bytree": trial.suggest_discrete_uniform("colsample_bytree", 0.6, 1.0, 0.05),  # Reduce the range


Optimizing hyperparameters...
Training Fold 1...
Fold 1 MAE: 41.44862986373071
Training Fold 2...
Fold 2 MAE: 37.13013677029474
Training Fold 3...
Fold 3 MAE: 40.7428083964109
Training Fold 4...
Fold 4 MAE: 36.555767248915984
Training Fold 5...
Fold 5 MAE: 39.97109747376991
Training Fold 6...
Fold 6 MAE: 41.348061816736376
Training Fold 7...
Fold 7 MAE: 39.559984123585416
Training Fold 8...
Fold 8 MAE: 38.97804534146955
Training Fold 9...
Fold 9 MAE: 40.27301714775857
Training Fold 10...


[I 2023-11-09 13:45:41,821] Trial 0 finished with value: 39.51513925696973 and parameters: {'tree_method': 'hist', 'max_depth': 3, 'learning_rate': 0.14953915054133893, 'n_estimators': 561, 'min_child_weight': 1, 'gamma': 0.6931954324404516, 'subsample': 0.8, 'colsample_bytree': 0.95, 'reg_alpha': 0.1535995730318329, 'reg_lambda': 0.27124951923885465, 'random_state': 659}. Best is trial 0 with value: 39.51513925696973.


Fold 10 MAE: 39.14384438702517
Mean MAE for all Folds: 39.51513925696973
Training Fold 1...


/var/folders/y8/9glnt_6110xftbsjpdqd12280000gp/T/ipykernel_3270/1140765383.py:16: FutureWarning: suggest_discrete_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., step=...) instead.
  "subsample": trial.suggest_discrete_uniform("subsample", 0.6, 1.0, 0.05),  # Reduce the range
/var/folders/y8/9glnt_6110xftbsjpdqd12280000gp/T/ipykernel_3270/1140765383.py:17: FutureWarning: suggest_discrete_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., step=...) instead.
  "colsample_bytree": trial.suggest_discrete_uniform("colsample_bytree", 0.6, 1.0, 0.05),  # Reduce the range


...

Fold 10 MAE: 37.33804915836515
Mean MAE for all Folds: 35.92698755019459
Training Fold 1...


/var/folders/y8/9glnt_6110xftbsjpdqd12280000gp/T/ipykernel_3270/1140765383.py:16: FutureWarning: suggest_discrete_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., step=...) instead.
  "subsample": trial.suggest_discrete_uniform("subsample", 0.6, 1.0, 0.05),  # Reduce the range
/var/folders/y8/9glnt_6110xftbsjpdqd12280000gp/T/ipykernel_3270/1140765383.py:17: FutureWarning: suggest_discrete_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., step=...) instead.
  "colsample_bytree": trial.suggest_discrete_uniform("colsample_bytree", 0.6, 1.0, 0.05),  # Reduce the range


Fold 1 MAE: 34.994183639842355
Training Fold 2...
Fold 2 MAE: 32.86541396773946
Training Fold 3...
Fold 3 MAE: 33.24690727861646
Training Fold 4...
Fold 4 MAE: 32.56099351892569
Training Fold 5...
Fold 5 MAE: 34.72467900352534
Training Fold 6...
Fold 6 MAE: 35.53915248419287
Training Fold 7...
Fold 7 MAE: 35.108060115809245
Training Fold 8...
Fold 8 MAE: 33.799266610586336
Training Fold 9...
Fold 9 MAE: 33.70400237232585
Training Fold 10...


[I 2023-11-09 14:01:22,052] Trial 29 finished with value: 34.061015356231266 and parameters: {'tree_method': 'hist', 'max_depth': 10, 'learning_rate': 0.09214226913891382, 'n_estimators': 559, 'min_child_weight': 9, 'gamma': 0.7108314281983517, 'subsample': 0.7, 'colsample_bytree': 0.75, 'reg_alpha': 0.1911678842656434, 'reg_lambda': 0.26606957916976376, 'random_state': 648}. Best is trial 27 with value: 33.24976193865119.


Fold 10 MAE: 34.06749457074909
Mean MAE for all Folds: 34.061015356231266
Optimization complete!


In [15]:
test1 = pipin.get_test_data("A")
test2 = pipin.get_test_data("B")
test3 = pipin.get_test_data("C")

test1 = pipin.drop_columns(test1)
test2 = pipin.drop_columns(test2)
test3 = pipin.drop_columns(test3)

In [16]:
pred1 = pd.DataFrame(predictor1.predict(test1), columns=["pv_measurement"])
pred2 = pd.DataFrame(predictor2.predict(test2), columns=["pv_measurement"])
pred3 = pd.DataFrame(predictor3.predict(test3), columns=["pv_measurement"])
negatives_pred1 = pred1[pred1["pv_measurement"] < 0]
negatives_pred2 = pred2[pred2["pv_measurement"] < 0]
negatives_pred3 = pred3[pred3["pv_measurement"] < 0]
neg = pd.concat([negatives_pred1, negatives_pred2, negatives_pred3])
neg.shape

(220, 1)

In [17]:
pred = pd.concat([pred1, pred2, pred3])
final_prediction = pipin.post_processing(pred, prediction_column="pv_measurement")
final_prediction.to_csv('submissions/XGB_143_pipeline.csv')

In [18]:
diff = pipin.compare_mae(final_prediction)
diff

61.68391307432472

In [19]:
# predictor.evaluate(df1, silent=True)